In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [ ]:
df = pd.read_csv("/Users/maurits/Documents/Ironhack/week_05/gpw5+6/vanguard-ab-test/data/raw/df_web_comb.csv")

In [7]:
df.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04


In [17]:
import pandas as pd
from datetime import datetime

# Assuming the data is already in a DataFrame called 'df'
# If not, you'd need to load it first, e.g.:
# df = pd.read_csv('your_data.csv')

# Convert date_time to datetime
df['date_time'] = pd.to_datetime(df['date_time'])

# Sort the dataframe by client_id and date_time
df = df.sort_values(['client_id', 'date_time'])

# Function to analyze process steps for a client
def analyze_client(group):
    steps = group['process_step'].tolist()
    times = group['date_time'].tolist()
    
    went_back = False
    reached_confirm = False
    total_duration = None
    
    if 'start' in steps and 'confirm' in steps:
        reached_confirm = True
        start_time = group[group['process_step'] == 'start']['date_time'].iloc[0]
        confirm_time = group[group['process_step'] == 'confirm']['date_time'].iloc[-1]
        total_duration = (confirm_time - start_time).total_seconds() / 60  # in minutes
    
    max_step = 0
    for step in steps:
        if step.startswith('step_'):
            current_step = int(step.split('_')[1])
            if current_step < max_step:
                went_back = True
                break
            max_step = max(max_step, current_step)
    
    return pd.Series({
        'went_back': went_back,
        'reached_confirm': reached_confirm,
        'total_duration_minutes': total_duration
    })

# Apply the analysis function to each client group
result = df.groupby('client_id').apply(analyze_client).reset_index()

# Display the results
display(result)

/var/folders/3v/mdv6bm7j6n99zg9hqgt9n70r0000gn/T/ipykernel_42220/1533984296.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby('client_id').apply(analyze_client).reset_index()


,client_id,went_back,reached_confirm,total_duration_minutes
0,169,False,True,3.550000
1,336,False,False,NaN
2,546,False,True,2.216667
3,555,False,True,2.633333
4,647,False,True,6.283333
...,...,...,...,...
120152,9999729,True,True,47668.850000
120153,9999768,True,True,8.100000
120154,9999832,False,False,NaN
120155,9999839,False,True,4.133333


In [ ]:
df_test = df[df['Variation'] == 'Test']
df_control = df[df['Variation'] == 'Control']


In [ ]:
df_test

In [ ]:
# Convert the 'full_process' column to a timedelta format
df['full_process'] = pd.to_timedelta(df['full_process'], errors='coerce')

# Now group by 'age_category' and calculate the mean of 'full_process'
age_category_mean_full_process = df.groupby('age_category')['full_process'].mean()

# Display the result
age_category_mean_full_process

In [ ]:
# Convert the 'full_process' column to a timedelta format
df['full_process'] = pd.to_timedelta(df['full_process'], errors='coerce')

# Now group by 'age_category' and calculate the mean of 'full_process'
age_category_mean_full_process = df.groupby('age_category')['full_process'].min()

# Display the result
age_category_mean_full_process

In [ ]:
# problematic_entries = df[df['full_process'].apply(lambda x: isinstance(x, pd.Timestamp) or isinstance(x, str))]['full_process'].unique()

# problematic_entries

In [ ]:
# df['full_process'] = df['full_process'].astype(str)
# df['full_process'] = pd.to_timedelta(df['full_process'], errors='coerce')

# # Check if the conversion is successful and there are no more errors
# df['full_process'].dtype, df['full_process'].head()

In [ ]:
# df['full_process'] = df['full_process'].astype(str)

In [ ]:
# df['full_process'] = pd.to_timedelta(df['full_process'], errors='coerce')


In [ ]:
# df.info()

In [ ]:
# df_test['full_process'].mean()

In [ ]:
df_test['full_process'] = pd.to_datetime(df['full_process'])
df_test_grouped = df_test.groupby('age_category')['full_process'].mean().reset_index()
df_test_grouped

In [ ]:
# Convert the 'start_time' and 'end_time' columns to datetime
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])

# Calculate time differences in seconds
df['completion_time_seconds'] = (df['end_time'] - df['start_time']).dt.total_seconds()

# Group by 'department' and calculate the mean completion time
grouped_df_completion_time = df.groupby('department')['completion_time_seconds'].mean().reset_index()

# Convert time from seconds to hours, if needed
grouped_df_completion_time['mean_completion_time_hours'] = grouped_df_completion_time['completion_time_seconds'] / 3600

print("\nGrouped DataFrame with Mean Completion Time (in seconds and hours):")
print(grouped_df_completion_time)